# 计算机是如何存储图片的

图片是由数字矩阵组成的，我们最常用的RGB格式的图片由三个通道的矩阵叠加而成 （H * W * C）


其实每个图片都由若干像素点构成，每个像素点存储了该位置的颜色，其实还隐藏存储了另一个信息，就是位置。每一个像素点都有一对索引元组，例如（1，1）就是第一行第一列的像素点，如果写作（1，1，255），就是说第一行第一列的像素点是黑色，以此类推，也就是这样一个矩阵就完整的记录着图片的所有信息


![](https://cdn-images-1.medium.com/max/1600/1*HPbsBA5BJovt-vrbF1C7Jg.png)

# 卷积 （Convolution）

白话解释一下啥叫卷积，很形象，就是“卷”，你假象一下，你把擦脸毛巾卷起来，成为一个圆柱体的卷，就是这个意思，抽象的说，就是将原来的对象变小一些，但又能保证原来图像中的信息尽量多的保留下来

![](img/conv.gif)

## 卷积核 & 滤波器 (kernel & filter)

![](https://cdn-images-1.medium.com/max/1600/1*TAo3aselJNVwrLLr654Myg.gif)

将图片的每个部分与滤波器进行矩阵运算，并将运算的值进行加总，形成新的特征图

![](https://cdn-images-1.medium.com/max/1600/1*32zCSTBi3giSApz1oQV-zA.gif)

我们可以添加多个卷积核，比如3个卷积核，可以学习3种不同的特征。 **卷积核需要与图像的通道数一致，当使用多卷积核时，其结果会堆叠在一起形成新的通道**

![](https://cdn-images-1.medium.com/max/1600/1*Ukb2msCjU3G5eS4a45f-lg.png)

# 跳跃 (Strid)

![](https://cdn-images-1.medium.com/max/1600/1*itcofCIVsGe7rBmciJcmVw.gif)

# 池化 (max-pooling)

池化层的作用是对图像进行降采样，如下图将 4x4的图片降采样为 2x2

![](https://4.bp.blogspot.com/-HCaeeLJ9PSI/WVpEpg-qWiI/AAAAAAAAWxM/v4Qg8BAHuxAaC9Eoe2JZWY8jdt_vyCy8gCLcBGAs/s1600/3591_2.PNG)

经过降采样虽然丢失了部分信息，但是依然保留了关键特征，方便网络更高效的计算

![](https://cloud.netlifyusercontent.com/assets/344dbf88-fdf9-42bb-adb4-46f01eedd629/b0a23aa8-e08a-44d2-8e59-c703cd485753/13-progressive-jpeg.gif)

# Mnist 大师版

![](https://2.bp.blogspot.com/-lo4ANmafiQM/WVpEeo68VPI/AAAAAAAAWxE/7FJBUGp-GrQFL5fUYEg6tQAJWXUHrtgWgCLcBGAs/s1600/3812_1.png)

In [1]:
from keras.datasets import mnist  
from keras.utils import np_utils  
import numpy as np

np.random.seed(10)  
  
# 读取minst数据 
(X_Train, y_Train), (X_Test, y_Test) = mnist.load_data()  
  
# 将数据转为4维(黑白图像只有灰度通道，所以最后一位是1)
X_Train4D = X_Train.reshape(X_Train.shape[0], 28, 28, 1).astype('float32')  
X_Test4D = X_Test.reshape(X_Test.shape[0], 28, 28, 1).astype('float32')

Using TensorFlow backend.


In [2]:
# 特征归一化
X_Train4D_norm = X_Train4D / 255  
X_Test4D_norm = X_Test4D /255  
  
# 标签one_hot化 
y_TrainOneHot = np_utils.to_categorical(y_Train)
y_TestOneHot = np_utils.to_categorical(y_Test)

In [6]:
from keras.models import Sequential  
from keras.layers import Dense,Dropout,Flatten,Conv2D,MaxPool2D 

model = Sequential()  
# Create CN layer 1  
model.add(Conv2D(filters=,  
                 kernel_size=(,),  
                 padding='same',  
                 input_shape=(,,),  
                 activation='relu',
                 name='conv2d_1')) 
# Create Max-Pool 1  
model.add(MaxPool2D(pool_size=(,), name='max_pooling2d_1'))  
  
# Create CN layer 2  
model.add(Conv2D(filters=,  
                 kernel_size=(,),  
                 padding='same',  
                 activation='relu',
                 name='conv2d_2'))  
  
# Create Max-Pool 2  
model.add(MaxPool2D(pool_size=(,), name='max_pooling2d_2'))  
  
# Add Dropout layer  
model.add(Dropout(0.25, name='dropout_1'))

In [7]:
# 最后将模型展平，并加上全连接层
model.add(Flatten(name='flatten_1'))
model.add(Dense(128, activation='relu', name='dense_1'))  
model.add(Dropout(0.5, name='dropout_2'))
model.add(Dense(10, activation='softmax', name='dense_2'))

In [8]:
model.summary()  

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 28, 28, 16)        416       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 16)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 14, 14, 36)        14436     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 7, 7, 36)          0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 7, 7, 36)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 1764)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               225920    
__________

In [9]:
# 编译模型
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])  
  
# 训练 
train_history = model.fit(x=X_Train4D_norm,  
                          y=y_TrainOneHot, validation_split=0.2,  
                          epochs=10, batch_size=300, verbose=1)

Train on 48000 samples, validate on 12000 samples
Epoch 1/10
48000/48000 [==============================] - 12s 240us/step - loss: 0.4840 - acc: 0.8488 - val_loss: 0.1036 - val_acc: 0.9696
Epoch 2/10
48000/48000 [==============================] - 2s 50us/step - loss: 0.1363 - acc: 0.9601 - val_loss: 0.0666 - val_acc: 0.9791
Epoch 3/10
48000/48000 [==============================] - 2s 50us/step - loss: 0.0990 - acc: 0.9699 - val_loss: 0.0527 - val_acc: 0.9838
Epoch 4/10
48000/48000 [==============================] - 2s 50us/step - loss: 0.0788 - acc: 0.9766 - val_loss: 0.0464 - val_acc: 0.9859
Epoch 5/10
48000/48000 [==============================] - 2s 50us/step - loss: 0.0681 - acc: 0.9789 - val_loss: 0.0412 - val_acc: 0.9870
Epoch 6/10
48000/48000 [==============================] - 2s 50us/step - loss: 0.0613 - acc: 0.9819 - val_loss: 0.0387 - val_acc: 0.9893
Epoch 7/10
48000/48000 [==============================] - 2s 50us/step - loss: 0.0533 - acc: 0.9839 - val_loss: 0.0344 - val_a